In [10]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-10-17
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-10-17
@Title : Tweepy Client

'''

'\n\n@Author: Vighnesh Harish Bilgi\n@Date: 2022-10-17\n@Last Modified by: Vighnesh Harish Bilgi\n@Last Modified time: 2022-10-17\n@Title : Tweepy Client\n\n'

In [19]:
import tweepy
import os
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

In [4]:
# os.environ['AWS_DEFAULT_REGION'] = 'ap-south-1'
# os.environ['AWS_ACCESS_KEY_ID'] = os.environ.get('test1_access_key')
# os.environ['AWS_SECRET_ACCESS_KEY'] = os.environ.get('test1_secret_access_key')
BEARER_TOKEN = os.environ.get('bearer_token')

# Create PySpark SparkSession
# spark = SparkSession.builder \
#     .master("local") \
#     .appName("Word Count on Text of Tweets") \
#     .getOrCreate()


In [24]:
def connect_to_tweepy():
    """

    Description:
        To connect to tweepy client
    Parameter:
        No arguments accepted
    Return:
        TweepyClient client

    """
    client = tweepy.Client(bearer_token=BEARER_TOKEN)
    return client

def fetch_tweets(client,query):
    """

    Description:
        To use tweepy client to fetch recent 100 tweets based on query.
    Parameter:
         TweepyClient client,
         string query
    Return:
        dictionary tweets

    """
    tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at','public_metrics'], max_results=100)
    return tweets   

def word_count(text):
    """

    Description:
        to count occurence of each word in a string and return result as a dictionary.
    Parameter:
        string text
    Return:
        dictionary counts

    """
    counts = dict()
    words = text.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

def tweets_to_dict(tweets):
    """

    Description:
        Retrieving values of tweets dictionary of certain field values and storing them and returning as a dictionary
    Parameter:
        dictionary tweets
    Return:
        dictionary results

    """
    results = []
    for tweet in tweets.data:
        if len(tweet.data) > 0:
            obj = {}
            obj['id'] = tweet.id
            obj['created_at'] = tweet.created_at
            obj['text'] = tweet.text
            obj['word_count_on_text'] = len(tweet.text.strip().split())
            obj['word_occurences_on_text'] = word_count(tweet.text)
            obj['retweet_count'] = tweet.public_metrics['retweet_count']
            obj['reply_count'] = tweet.public_metrics['reply_count']
            obj['like_count'] = tweet.public_metrics['like_count']
            obj['quote_count'] = tweet.public_metrics['quote_count']
            results.append(obj)
            # print(tweet.text)
            # if len(tweet.context_annotations) > 0:
                # print(tweet.context_annotations)

    return results

def dict_to_df(results):
    """

    Description:
        Converting dictionary argument from 'results' parameter to pandas dataframe.
    Parameter:
        dictionary results
    Return:
        DataFrame df

    """
    df = pd.DataFrame()
    for record in results:
        # print(record)
        df = df.append(record, ignore_index=True)

    return df

def main():

    client = connect_to_tweepy()

    hashtag = 'covid'
    query = f'#{hashtag} -is:retweet lang:en'

    tweets = fetch_tweets(client,query)

    results = tweets_to_dict(tweets)

    df = dict_to_df(results)

    print(df)
    # sparkDF=spark.createDataFrame(df) 
    # sparkDF.printSchema()
    # sparkDF.show()

    filename = f'{hashtag}_tweets.csv'
    df.to_csv(f"tweets_as_csv/{filename}", index=False)

    # sparkDF.coalesce(1)\
    #     .write\
    #     .format("csv")\
    #     .mode("overwrite")\
    #     .save("covid_tweets")

    # for tweet in tweets.data:
    #     if len(tweet.data) > 0:
    #             print(tweet)   

if __name__ == "__main__":
    main()



                     id                created_at  \
0   1593322359351496707 2022-11-17 19:17:07+00:00   
1   1593322194980831232 2022-11-17 19:16:28+00:00   
2   1593322177331437568 2022-11-17 19:16:24+00:00   
3   1593322172478812162 2022-11-17 19:16:23+00:00   
4   1593322168162598918 2022-11-17 19:16:22+00:00   
..                  ...                       ...   
95  1593310037346484224 2022-11-17 18:28:09+00:00   
96  1593309939358900224 2022-11-17 18:27:46+00:00   
97  1593309511401472002 2022-11-17 18:26:04+00:00   
98  1593309496868499456 2022-11-17 18:26:00+00:00   
99  1593309463930437633 2022-11-17 18:25:53+00:00   

                                                 text  word_count_on_text  \
0   @CSAC_Counties By the lack of masks, I guess #...                  21   
1   @genomicdao A very smart project brings us exc...                  30   
2   Japan minister quits over execution remark, PM...                  33   
3   Japan minister quits over execution remark, PM...  

C:\Users\Vighnesh\AppData\Local\Temp\ipykernel_23400\2358475419.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index=True)
C:\Users\Vighnesh\AppData\Local\Temp\ipykernel_23400\2358475419.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index=True)
C:\Users\Vighnesh\AppData\Local\Temp\ipykernel_23400\2358475419.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index=True)
C:\Users\Vighnesh\AppData\Local\Temp\ipykernel_23400\2358475419.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(record, ignore_index=True)
C:\Users